In [1]:
import hisepy as hp
import h5py
import pandas as pd
import scipy.sparse as scs
import os
import scanpy as sc
import anndata
import plotly

In [2]:
# Read CSV file
master = pd.read_csv("/home/jupyter/IH-A-Aging-Analysis-Notebooks_old/Mansi_Notebooks/scRNA/DEG_Analysis/MS_Sex_DEG_analysis/master_combined_DEG_list_all_celltypes.csv")
master.head()

,X,baseMean,log2FoldChange,lfcSE,stat,pvalue,padj,celltype,Direction,Year,type
0,NOC2L,0.468260,-0.335645,0.832711,-0.403074,0.686894,0.999827,Activated memory B cell,BR1,Y1,DEGs_Male
1,ISG15,0.771676,0.031020,0.532948,0.058204,0.953586,0.999827,Activated memory B cell,BR2,Y1,DEGs_Male
2,SDF4,0.563181,0.012470,0.651063,0.019153,0.984719,0.999827,Activated memory B cell,BR2,Y1,DEGs_Male
3,B3GALT6,0.315174,0.618455,0.877990,0.704399,0.481184,0.999827,Activated memory B cell,BR2,Y1,DEGs_Male
4,UBE2J2,0.451714,0.177616,0.652815,0.272078,0.785562,0.999827,Activated memory B cell,BR2,Y1,DEGs_Male


In [3]:
import pandas as pd

# Assuming 'master' is your DataFrame
# Filtered DataFrame where padj <= 0.1
filtered = master[master['padj'] <= 0.1]

print("Original DataFrame dimensions:", master.shape)
print("Filtered DataFrame dimensions:", filtered.shape)
print(filtered.head())

# Grouping and summarizing
filtered_df = filtered.groupby(['celltype', 'Direction', 'type']).size().reset_index(name='count')

print(filtered_df.head())
filtered_df.shape


Original DataFrame dimensions: (744822, 11)
Filtered DataFrame dimensions: (1559, 11)
             X     baseMean  log2FoldChange     lfcSE      stat    pvalue  \
5105     GTF2I    34.093557        0.346157  0.079471  4.355794  0.000013   
25481     ACTB  2853.100453       -0.201412  0.045960 -4.382339  0.000012   
40041   LGALS1    33.202331        0.878714  0.201613  4.358420  0.000013   
40042    APLP2     4.443854        0.926691  0.221221  4.188980  0.000028   
50505  PLEKHO1     3.451516       -1.048152  0.236010 -4.441142  0.000009   

           padj               celltype Direction Year       type  
5105   0.060857       Adaptive NK cell       BR2   Y1  DEGs_Male  
25481  0.080674     C1Q+ CD16 monocyte       BR1   Y1  DEGs_Male  
40041  0.060170  CD27- effector B cell       BR2   Y1  DEGs_Male  
40042  0.064350  CD27- effector B cell       BR2   Y1  DEGs_Male  
50505  0.047918               CD4 MAIT       BR1   Y1  DEGs_Male  
                celltype Direction         type  

(101, 4)

In [4]:
import pandas as pd

# Assuming 'master' is your DataFrame
# Filtered DataFrame where padj <= 0.1
filtered = master[master['padj'] <= 0.1]

print("Original DataFrame dimensions:", master.shape)
print("Filtered DataFrame dimensions:", filtered.shape)
print(filtered.head())

# Grouping and summarizing
filtered_df = filtered.groupby(['celltype', 'Direction', 'type']).size().reset_index(name='count')

# Add missing combinations and fill with 0
all_combinations = pd.MultiIndex.from_product([filtered['celltype'].unique(), 
                                               filtered['Direction'].unique(), 
                                               filtered['type'].unique()],
                                              names=['celltype', 'Direction', 'type'])

filtered_df = (filtered_df.set_index(['celltype', 'Direction', 'type'])
                          .reindex(all_combinations, fill_value=0)
                          .reset_index())

print(filtered_df.head())
filtered_df.shape

Original DataFrame dimensions: (744822, 11)
Filtered DataFrame dimensions: (1559, 11)
             X     baseMean  log2FoldChange     lfcSE      stat    pvalue  \
5105     GTF2I    34.093557        0.346157  0.079471  4.355794  0.000013   
25481     ACTB  2853.100453       -0.201412  0.045960 -4.382339  0.000012   
40041   LGALS1    33.202331        0.878714  0.201613  4.358420  0.000013   
40042    APLP2     4.443854        0.926691  0.221221  4.188980  0.000028   
50505  PLEKHO1     3.451516       -1.048152  0.236010 -4.441142  0.000009   

           padj               celltype Direction Year       type  
5105   0.060857       Adaptive NK cell       BR2   Y1  DEGs_Male  
25481  0.080674     C1Q+ CD16 monocyte       BR1   Y1  DEGs_Male  
40041  0.060170  CD27- effector B cell       BR2   Y1  DEGs_Male  
40042  0.064350  CD27- effector B cell       BR2   Y1  DEGs_Male  
50505  0.047918               CD4 MAIT       BR1   Y1  DEGs_Male  
             celltype Direction         type  cou

(152, 4)

In [5]:
filtered_df['celltype'].nunique()

38

In [7]:
#save the dataframe
filtered_df.to_csv("/home/jupyter/IH-A-Aging-Analysis-Notebooks_old/Mansi_Notebooks/scRNA/DEG_Analysis/MS_Sex_DEG_analysis/nDEGs_sourcedata.csv", index=False)


In [14]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go

In [15]:



# # Create dummy data indexed by month and with multi-columns [product, revenue]
# index = pd.date_range("2020", "2021", freq="MS", closed="left")
# df = pd.concat(
#     [
#         pd.DataFrame(
#             np.random.rand(12, 3) * 1.25 + 0.25,
#             index=index,
#             columns=["Revenue1", "Revenue2", "Revenue3"]
#         ),
#         pd.DataFrame(
#             np.random.rand(12, 3) + 0.5,
#             index=index,
#             columns=["Revenue1", "Revenue2", "Revenue3"]
#         ),
#     ],
#     axis=1,
#     keys=["Product1", "Product2"]
# )

# Create a figure with the right layout
fig = go.Figure(
    layout=go.Layout(
        height=600,
        width=1000,
        barmode="relative",
        yaxis_showticklabels=False,
        yaxis_showgrid=False,
        yaxis_range=[0, filtered_df.groupby(axis=1, level=0).sum().max().max() * 1.5],
       # Secondary y-axis overlayed on the primary one and not visible
        yaxis2=go.layout.YAxis(
            visible=False,
            matches="y",
            overlaying="y",
            anchor="x",
        ),
        font=dict(size=24),
        legend_x=0,
        legend_y=1,
        legend_orientation="h",
        hovermode="x",
        margin=dict(b=0,t=10,l=0,r=10)
    )
)

# Define some colors for the product, revenue pairs
colors = {
    "DEGs_Male": {
        "BR1": "#F28F1D",
        "BR2": "#F6C619",
    },
    "DEGs_Female": {
        "BR1": "#2B6045",
        "BR2": "#5EB88A",
    }
}

# Add the traces
for i, t in enumerate(colors):
    for j, col in enumerate(filtered_df[t].columns):
        if (filtered_df[t][col] == 0).all():
            continue
        fig.add_bar(
            x=filtered_df.celltype,
            y=filtered_df[t][col],
            # Set the right yaxis depending on the selected product (from enumerate)
            yaxis=f"y{i + 1}",
            # Offset the bar trace, offset needs to match the width
            # The values here are in milliseconds, 1billion ms is ~1/3 month
            offsetgroup=str(i),
            offset=(i - 1) * 1000000000,
            width=1000000000,
            legendgroup=t,
            legendgrouptitle_text=t,
            name=col,
            marker_color=colors[t][col],
            marker_line=dict(width=2, color="#333"),
            hovertemplate="%{y}<extra></extra>"
        )

fig.show()

/tmp/ipykernel_104142/1981512994.py:28: FutureWarning: DataFrame.groupby with axis=1 is deprecated. Do `frame.T.groupby(...)` without axis instead.
  yaxis_range=[0, filtered_df.groupby(axis=1, level=0).sum().max().max() * 1.5],


TypeError: '<=' not supported between instances of 'numpy.ndarray' and 'str'